In [6]:
import sys
import requests
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as pyplot
import os

# Electricity consumption forecast - next 24 hours: https://data.fingrid.fi/en/datasets/165

api_key = '3010957519b04fdc94439b784a78d4fe'

ts=datetime.now()

ts1=ts+relativedelta(hours=-24)
ts2=ts+relativedelta(hours=24)

# Creating start and end time variables
dt_str1 = ts1.strftime("%Y-%m-%dT%H:%M:%S.000Z")
dt_str2 = ts2.strftime("%Y-%m-%dT%H:%M:%S.000Z")

url='https://data.fingrid.fi/api/datasets/165/data'

headers ={'x-api-key': api_key}

params={'startTime': dt_str1, 'endTime': dt_str2, 'pageSize': 2000}

response = requests.get(url, headers=headers,params=params)
x = response.json()
ec_pred_df = pd.DataFrame(x['data'])
print(ec_pred_df)

     datasetId                 startTime                   endTime   value
0          165  2024-10-10T12:45:00.000Z  2024-10-10T13:00:00.000Z  9279.1
1          165  2024-10-10T12:30:00.000Z  2024-10-10T12:45:00.000Z  9283.8
2          165  2024-10-10T12:15:00.000Z  2024-10-10T12:30:00.000Z  9291.7
3          165  2024-10-10T12:00:00.000Z  2024-10-10T12:15:00.000Z  9304.0
4          165  2024-10-10T11:45:00.000Z  2024-10-10T12:00:00.000Z  9313.3
..         ...                       ...                       ...     ...
186        165  2024-10-08T14:15:00.000Z  2024-10-08T14:30:00.000Z  8999.1
187        165  2024-10-08T14:00:00.000Z  2024-10-08T14:15:00.000Z  8979.9
188        165  2024-10-08T13:45:00.000Z  2024-10-08T14:00:00.000Z  8967.7
189        165  2024-10-08T13:30:00.000Z  2024-10-08T13:45:00.000Z  8960.2
190        165  2024-10-08T13:15:00.000Z  2024-10-08T13:30:00.000Z  8951.0

[191 rows x 4 columns]


In [7]:
# Let's drop the first column
ec_pred_df = ec_pred_df.drop(['datasetId'], axis=1)

# Changing the names of the columns
ec_pred_columns = ['Aika', 'Kulutus (MW)']

if ec_pred_df.columns.tolist() != ec_pred_columns:
    ec_pred_df.columns = ['Ylimääräinen', 'Aika', 'Kulutus (MW)']

ec_pred_df = ec_pred_df[ec_pred_columns]

# Changing MW to kW
ec_pred_df['Kulutus (MW)'] = ec_pred_df['Kulutus (MW)'] * 1000

ec_pred_df.rename(columns={'Kulutus (MW)': 'Kulutus (kW)'}, inplace=True)

# Replacing unnecessary letters and numbers
ec_pred_df['Aika'] = ec_pred_df['Aika'].str.replace('T', ' ').str.replace('Z', '').str.replace('.000', '')

print(ec_pred_df.head())

                  Aika  Kulutus (kW)
0  2024-10-10 13:00:00     9279100.0
1  2024-10-10 12:45:00     9283800.0
2  2024-10-10 12:30:00     9291700.0
3  2024-10-10 12:15:00     9304000.0
4  2024-10-10 12:00:00     9313300.0


In [8]:
# Reversing the values
ec_pred_df['Aika'] = ec_pred_df['Aika'].values[::-1]
ec_pred_df['Kulutus (kW)'] = ec_pred_df['Kulutus (kW)'].values[::-1]

print(ec_pred_df)

                    Aika  Kulutus (kW)
0    2024-10-08 13:30:00     8951000.0
1    2024-10-08 13:45:00     8960200.0
2    2024-10-08 14:00:00     8967700.0
3    2024-10-08 14:15:00     8979900.0
4    2024-10-08 14:30:00     8999100.0
..                   ...           ...
186  2024-10-10 12:00:00     9313300.0
187  2024-10-10 12:15:00     9304000.0
188  2024-10-10 12:30:00     9291700.0
189  2024-10-10 12:45:00     9283800.0
190  2024-10-10 13:00:00     9279100.0

[191 rows x 2 columns]


In [10]:
# Making sure that all time values are datetime objects
ec_pred_df['Aika'] = pd.to_datetime(ec_pred_df['Aika'])

# Taking the hourly mean of electricity production
ec_pred_df.set_index('Aika', inplace=True)
ec_pred_df = ec_pred_df.resample('h').mean().reset_index()

# Restoring the columns
ec_pred_df = ec_pred_df[['Aika', 'Kulutus (kW)']]

# Removing the duplicates
ec_pred_df = ec_pred_df.loc[~ec_pred_df['Aika'].duplicated(keep='first')]

print(ec_pred_df)

                  Aika  Kulutus (kW)
0  2024-10-08 13:00:00     8955600.0
1  2024-10-08 14:00:00     8989250.0
2  2024-10-08 15:00:00     9009050.0
3  2024-10-08 16:00:00     9104875.0
4  2024-10-08 17:00:00     9189950.0
5  2024-10-08 18:00:00     9014625.0
6  2024-10-08 19:00:00     8868725.0
7  2024-10-08 20:00:00     8775550.0
8  2024-10-08 21:00:00     8558525.0
9  2024-10-08 22:00:00     8358125.0
10 2024-10-08 23:00:00     8191325.0
11 2024-10-09 00:00:00     8085425.0
12 2024-10-09 01:00:00     8062825.0
13 2024-10-09 02:00:00     8152950.0
14 2024-10-09 03:00:00     8511725.0
15 2024-10-09 04:00:00     8942650.0
16 2024-10-09 05:00:00     9166525.0
17 2024-10-09 06:00:00     9281000.0
18 2024-10-09 07:00:00     9322775.0
19 2024-10-09 08:00:00     9310550.0
20 2024-10-09 09:00:00     9304525.0
21 2024-10-09 10:00:00     9202600.0
22 2024-10-09 11:00:00     9117450.0
23 2024-10-09 12:00:00     9123650.0
24 2024-10-09 13:00:00     9130775.0
25 2024-10-09 14:00:00     9151025.0
2